# Projet

## Imports

In [1]:
import copy
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

2023-04-14 13:56:57.135426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Traitement données

## Build model M5

In [ ]:
# Modifier

model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(Conv1D(filters=128, kernel_size=80, strides=4, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(AvgPool1D())
model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('softmax'))  # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## Train model

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=10, validation_data=(x_test, y_test))

## Evaluate model on test dataset

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

## Evaluate model on small dataset

In [ ]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

## Save trained model

In [ ]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [ ]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [ ]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

## Generate C code for the trained model with 16-bit fixed-point representation

In [ ]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [ ]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv